In [13]:
import asyncio
import random
from functools import wraps


In [14]:

def retry_when_error(max_retries=4, base_delay=1, jitter=0.1):
    """
    A decorator to retry an asynchronous function when an exception occurs.

    Parameters:
    max_retries (int): The maximum number of retries before giving up. Default is 4.
    base_delay (int or float): The base delay between retries in seconds. Default is 1 second.
    jitter (float): The maximum random jitter to add to the delay to avoid thundering herd problem. Default is 0.1 seconds.

    Returns:
    function: A wrapped function that will be retried upon failure.

    Example:
    @retry_when_error(max_retries=3, base_delay=2, jitter=0.5)
    async def my_function():
        # function implementation
    """
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            retries = 0
            while retries < max_retries:
                try:
                    return await func(*args, **kwargs)
                except Exception as e:
                    if retries == max_retries - 1:
                        print(f"Failed after {retries + 1} retries")
                        raise
                    retries += 1
                    delay = (base_delay * 2 ** retries)  + (random.random() * jitter)
                    print(f"Retrying {retries}/{max_retries} with delay {delay:.3f} after error: {e}")
                    await asyncio.sleep(delay)
            return await func(*args, **kwargs)
        return wrapper
    return decorator


In [15]:
# test it out
@retry_when_error(jitter=60)
async def test(arg1, kwarg1=None):
    if random.random() < 0.5:
        raise Exception('test') 
    
    return f"with normal return {arg1} and {kwarg1}"

In [16]:
for i in range(10):
    print(f"trial {i} {await test('arg1', kwarg1='kwarg1')}")

Retrying 1/4 with delay 53.690 after error: test
trial 0 with normal return arg1 and kwarg1
trial 1 with normal return arg1 and kwarg1
trial 2 with normal return arg1 and kwarg1
Retrying 1/4 with delay 30.101 after error: test
Retrying 2/4 with delay 39.830 after error: test
Retrying 3/4 with delay 26.572 after error: test
trial 3 with normal return arg1 and kwarg1
trial 4 with normal return arg1 and kwarg1
trial 5 with normal return arg1 and kwarg1
Retrying 1/4 with delay 18.580 after error: test
Retrying 2/4 with delay 8.008 after error: test
trial 6 with normal return arg1 and kwarg1
trial 7 with normal return arg1 and kwarg1
trial 8 with normal return arg1 and kwarg1
trial 9 with normal return arg1 and kwarg1
